In [1]:
# modality interpolation
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import logging
import copy
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.neighbors import kneighbors_graph

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
# from utils.readers import InHospitalMortalityReader
# from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

In [2]:
import pickle

pdid = pickle.load(open("pdid.pkl", "rb"))

left_fundus = pickle.load(open("left_fundus.pkl", "rb"))
right_fundus = pickle.load(open("right_fundus.pkl", "rb"))

left_fundus_images = pickle.load(open("left_fundus_images.pkl", "rb"))
right_fundus_images = pickle.load(open("right_fundus_images.pkl", "rb"))

left_diag = pickle.load(open("left_diag.pkl", "rb"))
right_diag = pickle.load(open("right_diag.pkl", "rb"))

X = pickle.load(open("X.pkl", "rb"))
Y = pickle.load(open("Y.pkl", "rb"))

feature = pickle.load(open("feature.pkl", "rb"))


left_diag_mask = pickle.load(open("left_diag_mask.pkl", "rb"))
right_diag_mask = pickle.load(open("right_diag_mask.pkl", "rb"))


In [3]:
cls_left_diag = []
for i in range(len(left_diag)):
    cur_cur_dp = '[CLS] ' + left_diag[i]
    cls_left_diag.append(cur_cur_dp)
    
cls_right_diag = []
for i in range(len(right_diag)):
    cur_cur_dp = '[CLS] ' + right_diag[i]
    cls_right_diag.append(cur_cur_dp)

In [4]:
cls_left_diag_li = []
for i in cls_left_diag:
        
    cls_left_diag_li.append(i.split(' '))
    
cls_right_diag_li = []
for i in cls_right_diag:

        
    cls_right_diag_li.append(i.split(' '))

In [5]:
train_idx = pickle.load(open("train_idx",'rb'))
dev_idx = pickle.load(open("dev_idx",'rb'))
test_idx = pickle.load(open("test_idx",'rb'))


train_x = [X[i] for i in train_idx]
train_left_fundus_images = [left_fundus_images[i] for i in train_idx]
train_right_fundus_images = [right_fundus_images[i] for i in train_idx]

train_left_diag = [cls_left_diag_li[i] for i in train_idx]
train_right_diag = [cls_right_diag_li[i] for i in train_idx]
train_left_diag_mask = [left_diag_mask[i] for i in train_idx]
train_right_diag_mask = [right_diag_mask[i] for i in train_idx]

train_y = [Y[i] for i in train_idx]


dev_x = [X[i] for i in dev_idx]
dev_left_fundus_images = [left_fundus_images[i] for i in dev_idx]
dev_right_fundus_images = [right_fundus_images[i] for i in dev_idx]

dev_left_diag = [cls_left_diag_li[i] for i in dev_idx]
dev_right_diag = [cls_right_diag_li[i] for i in dev_idx]
dev_left_diag_mask = [left_diag_mask[i] for i in dev_idx]
dev_right_diag_mask = [right_diag_mask[i] for i in dev_idx]

dev_y = [Y[i] for i in dev_idx]


test_x = [X[i] for i in test_idx]
test_left_fundus_images = [left_fundus_images[i] for i in test_idx]
test_right_fundus_images = [right_fundus_images[i] for i in test_idx]

test_left_diag = [cls_left_diag_li[i] for i in test_idx]
test_right_diag = [cls_right_diag_li[i] for i in test_idx]
test_left_diag_mask = [left_diag_mask[i] for i in test_idx]
test_right_diag_mask = [right_diag_mask[i] for i in test_idx]


test_y = [Y[i] for i in test_idx]


print(sum(Y)/len(Y))
print(sum(train_y)/len(train_y))
# print(sum(big_test_Y)/len(big_test_Y))
print(sum(dev_y)/len(dev_y))
print(sum(test_y)/len(test_y))

[0.06142857 0.06057143 0.04685714]
[0.06107143 0.0625     0.0475    ]
[0.06571429 0.05428571 0.04857143]
[0.06       0.05142857 0.04      ]


In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cuda')
print("available device: {}".format(device))

available device: cuda:0


In [7]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision.transforms as T
# from torchvision.io import read_image


plt.rcParams["savefig.bbox"] = 'tight'
# torch.manual_seed(1)


In [8]:
def batch_iter(x, y, left_fundus, right_fundus, left_diag, left_diag_mask, right_diag, right_diag_mask, \
               batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size)  # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size]  # fetch out all the induces

        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx], left_fundus[idx], right_fundus[idx], left_diag[idx], \
                             left_diag_mask[idx], right_diag[idx], right_diag_mask[idx]))

        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_left_fundus = [e[2] for e in examples]
        batch_right_fundus = [e[3] for e in examples]
        batch_left_diag = [e[4] for e in examples]
        batch_left_diag_mask = [e[5] for e in examples]
        batch_right_diag = [e[6] for e in examples]
        batch_right_diag_mask = [e[7] for e in examples]
    

        yield batch_x, batch_y, batch_left_fundus, batch_right_fundus, batch_left_diag, \
        batch_right_diag,  [batch_left_diag_mask, batch_right_diag_mask]

In [9]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [10]:
from collections import namedtuple
import sys
from typing import List, Tuple, Dict, Set, Union
import torch
import torch.nn as nn
import torch.nn.utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import math, copy, time
from model_embeddings import ModelEmbeddings
# Hypothesis = namedtuple('Hypothesis', ['value', 'score'])
from torch.autograd import Variable

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)



class NMT_tran(nn.Module):
   

    def __init__(self, embed_size, hidden_size, vocab, dropout_rate=0.2):
        """ Init NMT Model.

        @param embed_size (int): Embedding size (dimensionality)
        @param hidden_size (int): Hidden Size (dimensionality)
        @param vocab (Vocab): Vocabulary object containing src and tgt languages
                              See vocab.py for documentation.
        @param dropout_rate (float): Dropout probability, for attention
        """
        super(NMT_tran, self).__init__()
        self.model_embeddings = ModelEmbeddings(hidden_size, vocab)
        self.hidden_size = hidden_size
        self.dropout_rate = dropout_rate
        self.vocab = vocab


      
        c = copy.deepcopy
        attn = MultiHeadedAttention(8, self.hidden_size)
        ff = PositionwiseFeedForward(self.hidden_size, self.hidden_size*4, self.dropout_rate)
        self.position = PositionalEncoding(embed_size, dropout_rate)
        self.encoder = Encoder(EncoderLayer(hidden_size, c(attn), c(ff), dropout_rate), 1)

#         self.high_encoder = Encoder(EncoderLayer(hidden_size, c(attn), c(ff), dropout_rate), 1)
       
        self.opt = nn.Linear(
            in_features=(hidden_size), out_features=1, bias=False)

        self.sigmoid = nn.Sigmoid()
      
        self.dropout = nn.Dropout(p=dropout_rate)
    

    def forward(self, source: List[List[str]]) -> torch.Tensor:
        
        # Compute sentence lengths
#         print(source)
        source_lengths = [len(s) for s in source]


        # Convert list of lists into tensors
        total_src_padded = self.vocab.src.to_input_tensor(
            source, device=self.device)   # Tensor: (src_len, b)
        

        
       
        enc_hiddens, first_hidden = self.encode(
            total_src_padded)


        
        
        return enc_hiddens, source_lengths


    
    def encode(self, source_padded: torch.Tensor) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """ Apply the encoder to source sentences to obtain encoder hidden states.
            Additionally, take the final states of the encoder and project them to obtain initial states for decoder.

        @param source_padded (Tensor): Tensor of padded source sentences with shape (src_len, b), where
                                        b = batch_size, src_len = maximum source sentence length. Note that
                                       these have already been sorted in order of longest to shortest sentence.
        @param source_lengths (List[int]): List of actual lengths for each of the source sentences in the batch
        @returns enc_hiddens (Tensor): Tensor of hidden units with shape (b, src_len, h*2), where
                                        b = batch size, src_len = maximum source sentence length, h = hidden size.
        @returns dec_init_state (tuple(Tensor, Tensor)): Tuple of tensors representing the decoder's initial
                                                hidden state and cell.
        """
        enc_hiddens, dec_init_state = None, None

        # print(source_padded.shape)
        source_padded = source_padded.permute(1,0) # b t 
#         print(source_padded.shape)
        src_mask = (source_padded != 0).unsqueeze(-2)

        X = self.model_embeddings.source(source_padded)

        
        enc_hiddens = self.encoder(X, src_mask) # b t h
        first_hidden = enc_hiddens[:,0,:]

       
        return enc_hiddens, first_hidden



    @property
    def device(self) -> torch.device:
        """ Determine which device to place the Tensors upon, CPU or GPU.
        """
        return self.model_embeddings.source.weight.device


In [11]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

def init_weights(module):
    if isinstance(module, (nn.Linear, nn.Embedding)):
        module.weight.data.normal_(mean=0.0, std=0.02)
    elif isinstance(module, nn.LayerNorm):
        module.bias.data.zero_()
        module.weight.data.fill_(1.0)

    if isinstance(module, nn.Linear) and module.bias is not None:
        module.bias.data.zero_()

In [12]:
from docopt import docopt
import numpy as np
from typing import List, Tuple, Dict, Set, Union
from tqdm import tqdm
from utils1 import read_corpus
from vocab import Vocab, VocabEntry

vocab = Vocab.build(cls_left_diag_li+cls_right_diag_li, 50000, 0)

initialize source vocabulary ..
number of word types: 135, number of word types w/ frequency >= 0: 135


In [13]:
class MM_transformer_encoder(nn.Module):
    def __init__(self, input_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(MM_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        #self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        #self.d_k = d_k  
        #self.d_v = d_v # the two can be equal
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.embed = nn.Linear(self.input_dim, self.d_model)

        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 5000)
       
        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)
        self.output = nn.Linear(self.d_model, self.output_dim)


        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

        #self.sparsemax = Sparsemax(dim=0)

    def forward(self, input, mask):
        # input shape [batch_size, timestep, feature_dim]
#         demographic = demo_input
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)
        assert(self.d_model % self.MHD_num_head == 0)

     
        input = self.embed(input)
#         print(input.shape)
        input = self.PositionalEncoding(input)# b t d_model
#         posi_input = embed_input# b t d_model

        
#         mask = subsequent_mask(time_step).to(device) # 1 t t 下三角
#         print(mask)
        contexts = self.SublayerConnection(input, lambda x: self.MultiHeadedAttention(input, input, input, mask))# b t d_model
        #contexts = self.MultiHeadedAttention(qs, ks, vs, mask)# b t h

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))# b t d_model
        
#         contexts = contexts.view(batch_size, 16 * time_step)

#
#         os = []
#         for j in range(contexts.shape[0]):
#             os.append(contexts[j,lens[j]-1])
            
#         os = torch.stack(os)
    
#         output = self.output(os)# b t 1
#         output = self.sigmoid(output)
          
        return contexts # b t h
    #, self.MultiHeadedAttention.attn

In [14]:
def euclidean_dist(x, y):
    b = x.size(0)
    xx = torch.pow(x, 2).sum(1, keepdim=True).expand(b, b)
    yy = torch.pow(y, 2).sum(1, keepdim=True).expand(b, b).t()
    dist = xx+yy-2*torch.mm(x, y.t())
    return dist 

def guassian_kernel(source, kernel_mul=2.0, kernel_num=1, fix_sigma=None):
    n = source.size(0)
    L2_distance = euclidean_dist(source, source)

        
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = torch.sum(L2_distance.data) / (n**2-n)
    
    if bandwidth < 1e-3:
        bandwidth = 1
    
    bandwidth /= kernel_mul ** (kernel_num//2)
    bandwidth_list = [bandwidth*(kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [torch.exp(-L2_distance/bandwidth_temp) for bandwidth_temp in bandwidth_list]
    return sum(kernel_val)/len(kernel_val)

In [15]:
# Our MAPLE framework
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
#         print(self.dropout)
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha# 4 leakyrelu
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.attention = None

    def forward(self, input, adj):# V N, V V
        h = torch.mm(input, self.W)# V O
        N = h.size()[0]# NUM OF V

        # V*V O ->123412341234, V*V O -> 111222333444, V V 2O
        a_input = torch.cat([h.repeat(1, N).view(N * N, -1), h.repeat(N, 1)], dim=1).view(N, -1, 2 * self.out_features)
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))# V V

        zero_vec = -9e15*torch.ones_like(e)# V V
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        self.attention = attention
        attention = F.dropout(attention, self.dropout, training=self.training)# V V
        h_prime = torch.matmul(attention, h)# V N

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime
        

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])
    
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(in_features, out_features).float())
        if bias:
            self.bias = Parameter(torch.Tensor(out_features).float())
        else:
            self.register_parameter('bias', None)
        self.initialize_parameters()

    def initialize_parameters(self):
        std = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-std, std)
        if self.bias is not None:
            self.bias.data.uniform_(-std, std)

    def forward(self, adj, x):
        y = torch.mm(x.float(), self.weight.float())
        output = torch.mm(adj.float(), y.float())
        if self.bias is not None:
            return output + self.bias.float()
        else:
            return output

In [16]:
import torchvision.models as models

class M3Care(nn.Module):
    def __init__(self, input_dim, hidden_dim, embed_size, output_dim=1, keep_prob=1):
        super(M3Care, self).__init__()

        # hyperparameters
        self.input_dim = input_dim 
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.keep_prob = keep_prob
        self.modal_num = 3
        
        self.NLP_model = NMT_tran(embed_size=embed_size,
                hidden_size=hidden_dim,
                dropout_rate= 1 - self.keep_prob,
                vocab=vocab).to(device)
        
        
        
        self.linear1 = nn.Linear(self.input_dim, self.hidden_dim)
        
        self.res2hidden1 = nn.Linear(1000, self.hidden_dim)
#         self.res2hidden1 = nn.Linear(1000, self.hidden_dim)
#         self.linear_s = nn.Linear(28, self.hidden_dim)
#         self.linear2 = nn.Linear(self.hidden_dim *5, self.output_dim)
        self.MM_model = MM_transformer_encoder(input_dim = self.hidden_dim, d_model = self.hidden_dim,  \
                                                      MHD_num_head = 4 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)
        self.MM_model2 = MM_transformer_encoder(input_dim = self.hidden_dim, d_model = self.hidden_dim,  \
                                                      MHD_num_head = 1 , d_ff = self.hidden_dim*4, output_dim = 1).to(device)


        self.token_type_embeddings = nn.Embedding(6, self.hidden_dim)
        self.token_type_embeddings.apply(init_weights)
        
        
        self.sep_token_embeddings = nn.Embedding(6, self.hidden_dim)
        self.sep_token_embeddings.apply(init_weights)
        
        self.PositionalEncoding = PositionalEncoding(self.hidden_dim, dropout = 0, max_len = 5000)
#         emb_data = self.token_type_embeddings.weight.data
#         self.token_type_embeddings = nn.Embedding(3, hs)
#         self.token_type_embeddings.apply(objectives.init_weights)
#         self.token_type_embeddings.weight.data[0, :] = emb_data[0, :]
#         self.token_type_embeddings.weight.data[1, :] = emb_data[1, :]
#         self.token_type_embeddings.weight.data[2, :] = emb_data[1, :]

        
        
        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax(-1)
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()
        
        self.proj1 = nn.Linear(self.hidden_dim *5, self.hidden_dim*2 )
        self.proj2 = nn.Linear(self.hidden_dim *5, self.hidden_dim *5)
        self.out_layer = nn.Linear(self.hidden_dim *2, self.output_dim)
        
        
        
        self.threshold = nn.Parameter(torch.ones(size=(1,))+1)
        self.simiProj = nn.Linear(self.hidden_dim, self.hidden_dim)
        
        
        self.resnet18 =  models.resnet18()
        
        self.selu=nn.SELU()
        
    
        self.bn = nn.BatchNorm1d(self.hidden_dim)
    
        self.simiProj = clones(torch.nn.Sequential(
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
            self.relu,
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
            self.relu,
            torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
#             torch.nn.Softplus(),
#             torch.nn.Linear(self.hidden_dim, self.hidden_dim, bias=True),
        ), self.modal_num)
        
        self.GCN = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        self.GCN_2 = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        self.GCN_3 = clones(GraphConvolution(self.hidden_dim, self.hidden_dim, bias=True),self.modal_num)
        
        self.eps0 = nn.Parameter(torch.ones(size=(1,))+1)
        self.eps1 = nn.Parameter(torch.ones(size=(1,))+1)
        self.eps2 = nn.Parameter(torch.ones(size=(1,))+1)

        
        self.weight1 = clones(nn.Linear(self.hidden_dim, 1), self.modal_num)
        self.weight2 = clones(nn.Linear(self.hidden_dim, 1), self.modal_num)

    def forward(self, input, left_fundus, right_fundus, left_diag, right_diag, l_r_masks):
        
        
        values_hidden = self.relu(self.linear1(input))# b 1
        val_mask = length_to_mask(torch.ones((values_hidden.shape[0],1)).int().squeeze()).unsqueeze(1).to(device).int()
       
#         left_diag_contexts = torch.cat([self.sep_token_embeddings(torch.zeros_like(left_diag_lens).to(device).long()).unsqueeze(1), pd_contexts], dim=1)
#         left_diag_mask = torch.cat([torch.ones_like(left_diag_lens).to(device).unsqueeze(-1).unsqueeze(-1), left_diag_mask.int()], dim=-1)
        
        
        
#         print(left_diag)
        left_diag_contexts, left_diag_lens = self.NLP_model(left_diag)# b t h
#         left_diag_lens = torch.tensor(left_diag_lens, dtype=torch.float32).to(device).int()
        
        left_diag_contexts = self.relu(left_diag_contexts)
        left_diag_mask = length_to_mask(torch.from_numpy(np.array(left_diag_lens))).unsqueeze(1).to(device)
        
        
            
            
#         print(right_diag)
        right_diag_contexts, right_diag_lens = self.NLP_model(right_diag)# b t h
        right_diag_contexts = self.relu(right_diag_contexts)
        right_diag_mask = length_to_mask(torch.from_numpy(np.array(right_diag_lens))).unsqueeze(1).to(device)
        
        
        
        left_f = self.relu(self.res2hidden1(self.resnet18(left_fundus)))
        left_f_mask = length_to_mask(torch.ones((values_hidden.shape[0],1)).int().squeeze()).unsqueeze(1).to(device).int()
        
        
        
        right_f = self.relu(self.res2hidden1(self.resnet18(right_fundus)))
        right_f_mask = length_to_mask(torch.ones((values_hidden.shape[0],1)).int().squeeze()).unsqueeze(1).to(device).int()
        
        
        
        values_hidden00 = values_hidden
        left_diag_hidden00 = torch.zeros_like(left_diag_contexts[:, 0 ])
        right_diag_hidden00 = torch.zeros_like(left_diag_contexts[:, 0 ])
        left_f_hidden00 = torch.zeros_like(left_diag_contexts[:, 0 ])
        right_f_hidden00 = torch.zeros_like(left_diag_contexts[:, 0 ])
        for j in range(values_hidden00.shape[0]):
            left_diag_hidden00[j] = left_diag_contexts[j, 0 ]
            right_diag_hidden00[j] = right_diag_contexts[j, 0]
            left_f_hidden00[j] = left_f[j]
            right_f_hidden00[j] = right_f[j]
            
#         left_diag_hidden00 = torch.stack(left_diag_hidden0)
#         right_diag_hidden00 = torch.stack(right_diag_hidden0)
#         left_f_hidden00 = torch.stack(left_f_hidden0)
#         right_f_hidden00 = torch.stack(right_f_hidden0)
        
        
        left_diag_mask_ = torch.from_numpy(np.array(l_r_masks[0])).to(device).unsqueeze(1)# b 1
        right_diag_mask_ = torch.from_numpy(np.array(l_r_masks[1])).to(device).unsqueeze(1)
        
        left_diag_mask2 = left_diag_mask_ * left_diag_mask_.permute(1,0)
        right_diag_mask2 = right_diag_mask_ * right_diag_mask_.permute(1,0)
        
        
        values_hidden_mat = guassian_kernel(self.bn(self.relu(self.simiProj[0](values_hidden00))), kernel_mul=2.0, kernel_num=3)
        values_hidden_mat2 = guassian_kernel(self.bn(values_hidden00), kernel_mul=2.0, kernel_num=3)
        values_hidden_mat = ((1-self.sigmoid(self.eps0))*values_hidden_mat+self.sigmoid(self.eps0))*values_hidden_mat2
#         values_hidden_mat = values_hidden_mat*dp_mask2

        
        left_diag_hidden_mat = guassian_kernel(self.bn(self.relu(self.simiProj[1](left_diag_hidden00))), kernel_mul=2.0, kernel_num=3)
        left_diag_hidden_mat2 = guassian_kernel(self.bn(left_diag_hidden00), kernel_mul=2.0, kernel_num=3)
        left_diag_hidden_mat = ((1-self.sigmoid(self.eps1))*left_diag_hidden_mat+self.sigmoid(self.eps1))*left_diag_hidden_mat2
        left_diag_hidden_mat = left_diag_hidden_mat*left_diag_mask2
        
        
        right_diag_hidden_mat = guassian_kernel(self.bn(self.relu(self.simiProj[1](right_diag_hidden00))), kernel_mul=2.0, kernel_num=3)
        right_diag_hidden_mat2 = guassian_kernel(self.bn(right_diag_hidden00), kernel_mul=2.0, kernel_num=3)
        right_diag_hidden_mat = ((1-self.sigmoid(self.eps1))*right_diag_hidden_mat+self.sigmoid(self.eps1))*right_diag_hidden_mat2
        right_diag_hidden_mat = right_diag_hidden_mat*right_diag_mask2
    

    
        left_f_hidden_mat = guassian_kernel(self.bn(self.relu(self.simiProj[2](left_f_hidden00))), kernel_mul=2.0, kernel_num=3)
        left_f_hidden_mat2 = guassian_kernel(self.bn(left_f_hidden00), kernel_mul=2.0, kernel_num=3)
        left_f_hidden_mat = ((1-self.sigmoid(self.eps2))*left_f_hidden_mat+self.sigmoid(self.eps2))*left_f_hidden_mat2
        
        right_f_hidden_mat = guassian_kernel(self.bn(self.relu(self.simiProj[2](right_f_hidden00))), kernel_mul=2.0, kernel_num=3)
        right_f_hidden_mat2 = guassian_kernel(self.bn(right_f_hidden00), kernel_mul=2.0, kernel_num=3)
        right_f_hidden_mat = ((1-self.sigmoid(self.eps2))*right_f_hidden_mat+self.sigmoid(self.eps2))*right_f_hidden_mat2
        
        diff1 = torch.abs(torch.norm(self.simiProj[0](values_hidden00)) - torch.norm(values_hidden00))
        diff2 = torch.abs(torch.norm(self.simiProj[1](left_diag_hidden00)) - torch.norm(left_diag_hidden00))
        diff22 = torch.abs(torch.norm(self.simiProj[1](right_diag_hidden00)) - torch.norm(right_diag_hidden00))
        
        diff3 = torch.abs(torch.norm(self.simiProj[2](left_f_hidden00)) - torch.norm(left_f_hidden00))
        diff33 = torch.abs(torch.norm(self.simiProj[2](right_f_hidden00)) - torch.norm(right_f_hidden00))
        
        sum_of_diff = diff1+diff2+diff22+diff3+diff33
        
        similar_score = (values_hidden_mat+ left_diag_hidden_mat + right_diag_hidden_mat + left_f_hidden_mat +\
                         right_f_hidden_mat)/ \
        (torch.ones_like(right_diag_mask2)+ torch.ones_like(right_diag_mask2) + torch.ones_like(right_diag_mask2) \
         + right_diag_mask2 +  left_diag_mask2 )
        


        similar_score = self.relu(similar_score - self.sigmoid(self.threshold)[0])  
        temp_thresh = self.sigmoid(self.threshold)[0]
        bin_mask = similar_score>0
        similar_score = similar_score + bin_mask * temp_thresh.detach()

 
        
        left_diag_hidden0 = self.relu(self.GCN[0](similar_score*left_diag_mask2, left_diag_hidden00))
        left_diag_hidden0 = self.relu(self.GCN_2[0](similar_score*left_diag_mask2, left_diag_hidden0))

        right_diag_hidden0 = self.relu(self.GCN[1](similar_score*right_diag_mask2, right_diag_hidden00))
        right_diag_hidden0 = self.relu(self.GCN_2[1](similar_score*right_diag_mask2, right_diag_hidden0))  
        
        
        left_diag_weight1=torch.sigmoid(self.weight1[0](left_diag_hidden0))
        left_diag_weight2 = torch.sigmoid(self.weight2[0](left_diag_hidden00 ))
        left_diag_weight1 = left_diag_weight1/(left_diag_weight1+left_diag_weight2)
        left_diag_weight2= 1-left_diag_weight1
        
        right_diag_weight1=torch.sigmoid(self.weight1[1](right_diag_hidden0))
        right_diag_weight2 = torch.sigmoid(self.weight2[1](right_diag_hidden00 ))
        right_diag_weight1 = right_diag_weight1/(right_diag_weight1+right_diag_weight2)
        right_diag_weight2= 1-right_diag_weight1
        
        

        final_left_diag = left_diag_weight1*left_diag_hidden0+left_diag_weight2*left_diag_hidden00
        final_right_diag = right_diag_weight1*right_diag_hidden0+right_diag_weight2*right_diag_hidden00
        
        
        right_diag_contexts_ = torch.zeros_like(right_diag_contexts)
        for i in range(values_hidden00.shape[0]):
            right_diag_contexts_[i] = right_diag_contexts[i]
            if right_diag_mask_[i][0] != 1:
                right_diag_contexts_[i,0] = right_diag_hidden0[i]

            else:
                 right_diag_contexts_[i,0] = final_right_diag[i]
                    
        left_diag_contexts_ = torch.zeros_like(left_diag_contexts)
        for i in range(values_hidden00.shape[0]):
            left_diag_contexts_[i] = left_diag_contexts[i]
            if left_diag_mask_[i][0] != 1:
                left_diag_contexts_[i,0] = left_diag_hidden0[i]

            else:
                 left_diag_contexts_[i,0] = final_left_diag[i]
        

                

        left_diag_contexts = self.PositionalEncoding(left_diag_contexts_)  +  \
            self.token_type_embeddings(torch.ones_like(left_diag_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         left_diag_contexts = torch.cat([self.sep_token_embeddings(torch.zeros_like(torch.from_numpy(np.array(left_diag_lens))).to(device).long()).unsqueeze(1), left_diag_contexts], dim=1)
#         left_diag_mask = torch.cat([torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).int().to(device).unsqueeze(-1).unsqueeze(-1), left_diag_mask.int()], dim=-1)
        
        
        values_hidden = values_hidden.unsqueeze(1)  +  \
            self.token_type_embeddings(torch.zeros_like(val_mask.permute(0,2,1).squeeze(-1)).to(device).long())

        
        right_diag_contexts = self.PositionalEncoding(right_diag_contexts_)  +  \
            self.token_type_embeddings(2*torch.ones_like(right_diag_mask.permute(0,2,1).squeeze(-1)).to(device).long())

#         right_diag_contexts = torch.cat([self.sep_token_embeddings(torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).to(device).long()).unsqueeze(1), right_diag_contexts], dim=1)
#         right_diag_mask = torch.cat([torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).int().to(device).unsqueeze(-1).unsqueeze(-1), right_diag_mask.int()], dim=-1)
        
        left_f_contexts = left_f.unsqueeze(1)  +  \
            self.token_type_embeddings(3* torch.ones_like(left_f_mask.permute(0,2,1).squeeze(-1)).to(device).long())
#         left_f_contexts = torch.cat([self.sep_token_embeddings(2* torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).to(device).long()).unsqueeze(1), left_f], dim=1)
#         left_f_mask = torch.cat([torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).int().to(device).unsqueeze(-1).unsqueeze(-1), left_f_mask.int()], dim=-1)
        
        
        right_f_contexts = right_f.unsqueeze(1)  +  \
            self.token_type_embeddings(4* torch.ones_like(right_f_mask.permute(0,2,1).squeeze(-1)).to(device).long())
#         right_f_contexts = torch.cat([self.sep_token_embeddings(3* torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).to(device).long()).unsqueeze(1), right_f], dim=1)
#         right_f_mask = torch.cat([torch.ones_like(torch.from_numpy(np.array(left_diag_lens))).int().to(device).unsqueeze(-1).unsqueeze(-1), right_f_mask.int()], dim=-1)
           
        
        
        z0 = torch.cat([values_hidden, left_diag_contexts, right_diag_contexts, left_f_contexts, \
                        right_f_contexts], dim=1)
        z0_mask = torch.cat([val_mask.int(), left_diag_mask.int(), right_diag_mask.int(), left_f_mask.int(), \
                             right_f_mask.int()], dim=-1).int()
        
        z1 = self.relu(self.MM_model(z0, z0_mask))# b 1


        z2 = self.relu(self.MM_model2(z1, z0_mask))# b 1
    
    
        val_hidden = z2[:,0,:]
        left_diag_hidden = []
        right_diag_hidden = []
        left_f_hidden = []
        right_f_hidden = []
        for j in range(z2.shape[0]):
            left_diag_hidden.append(z2[j,1  ])
            right_diag_hidden.append(z2[j,1  + left_diag_contexts.shape[1]])
            left_f_hidden.append(z2[j,1 +left_diag_contexts.shape[1] + right_diag_contexts.shape[1] ])
            right_f_hidden.append(z2[j,1 +left_diag_contexts.shape[1] + right_diag_contexts.shape[1] \
                                    + left_f_contexts.shape[1]])
            
        left_diag_hidden = torch.stack(left_diag_hidden)
        right_diag_hidden = torch.stack(right_diag_hidden)
        left_f_hidden = torch.stack(left_f_hidden)
        right_f_hidden = torch.stack(right_f_hidden)

    
    
        


        
        combined_hidden = torch.cat((val_hidden,left_diag_hidden,right_diag_hidden,left_f_hidden, \
                                     right_f_hidden),-1)#b n h


        
        last_hs_proj = self.dropout(F.relu(self.proj1(combined_hidden)))

        
        output = self.out_layer(last_hs_proj).squeeze(-1)

        return output, sum_of_diff

In [17]:
def get_loss(y_pred, y_true, weight=None):
    loss = torch.nn.BCEWithLogitsLoss(weight=weight)
    return loss(y_pred, y_true)

In [18]:
transforms = T.Compose([ # C H W input shape needed
            T.ToPILImage(),
#             T.Resize(size = (224,224)),
#             T.Resize(255),  # We use single int value inside a list due to torchscript type restrictions
            T.CenterCrop(1728),
            T.Resize(size = (224,224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [19]:
IMAGE_PATH = "./archive"

In [20]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn

epochs = 100
batch_size = 512

fold_count = 0
total_train_loss = []
total_valid_loss = []
global_best = 0
_global_best = 0



history = []

        
model = M3Care(input_dim = X.shape[-1], hidden_dim = 128, embed_size = 128, output_dim = 3, keep_prob=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay = 1e-4)

    

file_name = './model/M3Care-model2' 

fold_train_loss = []
fold_valid_loss = []

best_auc_scores = 0
best_ave_auc_micro = 0
best_ave_auc_macro = 0
best_coverage_error = 0
best_label_ranking_loss = 0


_best_auc_scores = 0
_best_ave_auc_micro = 0
_best_ave_auc_macro = 0
_best_coverage_error = 0
_best_label_ranking_loss = 0

for each_epoch in range(epochs):

    epoch_loss = []
    counter_batch = 0
    model.train()  
#         print(0)
    for step, (batch_x, batch_y, batch_left_fundus_images, batch_right_fundus_images, batch_left_diag, \
    batch_right_diag, l_r_masks) in enumerate(
            batch_iter(train_x, train_y, train_left_fundus_images, train_right_fundus_images, train_left_diag,
                        train_left_diag_mask, train_right_diag, train_right_diag_mask, \
                       batch_size, shuffle=True)):
        optimizer.zero_grad()

        batch_x = torch.tensor(batch_x, dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device).squeeze(-1)

        batch_left_fundus_images = torch.tensor(batch_left_fundus_images, dtype=torch.float32).to(device)
        batch_right_fundus_images = torch.tensor(batch_right_fundus_images, dtype=torch.float32).to(device)


        opt, sum_of_diff= model(batch_x, batch_left_fundus_images, batch_right_fundus_images, \
                    batch_left_diag, batch_right_diag, l_r_masks)
        
        


        BCE_Loss = get_loss(opt, batch_y)
#         print(BCE_Loss, sum_of_diff*1e-5)
        model_loss =  BCE_Loss 
        loss = model_loss   + sum_of_diff*1e-7


        epoch_loss.append(BCE_Loss.cpu().detach().numpy())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
        optimizer.step()


    epoch_loss = np.mean(epoch_loss)
    fold_train_loss.append(epoch_loss)

    #Validation
    y_true = []
    y_pred = []

    with torch.no_grad():
        model.eval()
        valid_loss = []
        valid_true = []
        valid_pred = []

        for step, (batch_x, batch_y, batch_left_fundus_images, batch_right_fundus_images, batch_left_diag, \
    batch_right_diag, l_r_masks) in enumerate(
            batch_iter(dev_x, dev_y, dev_left_fundus_images, dev_right_fundus_images, dev_left_diag,
                        dev_left_diag_mask, dev_right_diag, dev_right_diag_mask, batch_size)):

            batch_x = torch.tensor(batch_x, dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device).squeeze(-1)

            batch_left_fundus_images = torch.tensor(batch_left_fundus_images, dtype=torch.float32).to(device)
            batch_right_fundus_images = torch.tensor(batch_right_fundus_images, dtype=torch.float32).to(device)


            opt, sum_of_diff = model(batch_x, batch_left_fundus_images, batch_right_fundus_images, \
                        batch_left_diag, batch_right_diag, l_r_masks)


            BCE_Loss = get_loss(opt, batch_y)

            valid_loss.append(BCE_Loss.cpu().detach().numpy())


            y_pred += list(F.sigmoid(opt).cpu().detach().numpy())
            y_true += list(batch_y.cpu().numpy())



        valid_loss = np.mean(valid_loss)
        fold_valid_loss.append(valid_loss)

        ret = metrics.print_metrics_multilabel(y_true, y_pred,verbose = 0)

        history.append(ret)

        cur_ave_auc_micro = ret['ave_auc_micro']
        if cur_ave_auc_micro > best_ave_auc_micro:
            best_y_true = y_true
            best_y_pred = y_pred

            best_ave_auc_micro = cur_ave_auc_micro

            best_auc_scores = ret['auc_scores']
            best_ave_auc_macro = ret['ave_auc_macro']
            best_coverage_error = ret['coverage_error']
            best_label_ranking_loss = ret['label_ranking_loss']

            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name)


            print('------------ Save best model - ave_auc_micro: %.4f ------------'%cur_ave_auc_micro)

        cur_ave_auc_macro = ret['ave_auc_macro']
        

    print('Fold %d, Epoch %d, ave_auc_micro = %.4f, ave_auc_macro = %.4f, coverage_error = %.4f' \
          %(fold_count, each_epoch, ret['ave_auc_micro'], ret['ave_auc_macro'], ret['coverage_error']))



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


------------ Save best model - ave_auc_micro: 0.4554 ------------
Fold 0, Epoch 0, ave_auc_micro = 0.4554, ave_auc_macro = 0.3945, coverage_error = 0.3343


/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


------------ Save best model - ave_auc_micro: 0.5387 ------------
Fold 0, Epoch 1, ave_auc_micro = 0.5387, ave_auc_macro = 0.5415, coverage_error = 0.3200


/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


...

/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Fold 0, Epoch 97, ave_auc_micro = 0.7179, ave_auc_macro = 0.7140, coverage_error = 0.2543


/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Fold 0, Epoch 98, ave_auc_micro = 0.7138, ave_auc_macro = 0.7094, coverage_error = 0.2514
Fold 0, Epoch 99, ave_auc_micro = 0.7157, ave_auc_macro = 0.7094, coverage_error = 0.2629


/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [21]:
checkpoint = torch.load(file_name+'')# try bmm
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

last saved model is in epoch 20


M3Care(
  (NLP_model): NMT_tran(
    (model_embeddings): ModelEmbeddings(
      (source): Embedding(139, 128, padding_idx=0)
    )
    (position): PositionalEncoding(
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (encoder): Encoder(
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): MultiHeadedAttention(
            (linears): ModuleList(
              (0): Linear(in_features=128, out_features=128, bias=True)
              (1): Linear(in_features=128, out_features=128, bias=True)
              (2): Linear(in_features=128, out_features=128, bias=True)
              (3): Linear(in_features=128, out_features=128, bias=True)
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (feed_forward): PositionwiseFeedForward(
            (w_1): Linear(in_features=128, out_features=512, bias=True)
            (w_2): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.5, inplace=False)
      

In [22]:
batch_loss = [] 
y_true = []
y_pred = []
opts = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_left_fundus_images, batch_right_fundus_images, batch_left_diag, \
    batch_right_diag, l_r_masks) in enumerate(
            batch_iter(test_x, test_y, test_left_fundus_images, test_right_fundus_images, test_left_diag,
                        test_left_diag_mask, test_right_diag, test_right_diag_mask, batch_size)):

        batch_x = torch.tensor(batch_x, dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device).squeeze(-1)

        batch_left_fundus_images = torch.tensor(batch_left_fundus_images, dtype=torch.float32).to(device)
        batch_right_fundus_images = torch.tensor(batch_right_fundus_images, dtype=torch.float32).to(device)


        opt, sum_of_diff = model(batch_x, batch_left_fundus_images, batch_right_fundus_images, \
                    batch_left_diag, batch_right_diag, l_r_masks)

        BCE_Loss = get_loss(opt, batch_y)
        print(BCE_Loss)
#                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

        batch_loss.append(BCE_Loss.cpu().detach().numpy())

        opts += list(opt.cpu().detach().numpy())
        y_pred += list(F.sigmoid(opt).cpu().detach().numpy())
        y_true += list(batch_y.cpu().numpy())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
# y_pred = np.array(y_pred)
# y_pred = np.stack([1 - y_pred, y_pred], axis=1)
ret = metrics.print_metrics_multilabel(y_true, y_pred, verbose = 0)
for k, v in ret.items():
    print(k, v.round(5))
# print(ret)

tensor(0.1484, device='cuda:0')

==>Predicting on test
Test Loss = 0.1484
auc_scores [0.75771 0.97875 0.75978]
ave_auc_micro 0.84934
ave_auc_macro 0.83208
ave_auc_weighted 0.83333
ave_f1_micro 0.29851
ave_f1_macro 0.20833
coverage_error 0.19714
label_ranking_loss 0.02286


/home/zch/.local/lib/python3.6/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [23]:
# Bootstrap
N = len(y_true)
N_idx = np.arange(N)
K = 1000

y_true = np.array(y_true)
y_pred = np.array(y_pred)
opts = np.array(opts)

auc_micro = []
auc_macro = []
auc_weighted = []
coverage_error = []
test_loss = []
label_ranking_loss = []

for i in range(K):
    boot_idx = np.random.choice(N_idx, N, replace=True)
    boot_true = np.array(y_true)[boot_idx]
    boot_pred = y_pred[boot_idx, :]
    boot_opt = opts[boot_idx, :]
    
    BCE_Loss = get_loss(torch.tensor(boot_opt), torch.tensor(boot_true)).detach().numpy()
    test_loss.append(BCE_Loss)
    
    test_ret = metrics.print_metrics_multilabel(boot_true, boot_pred, verbose=0)
    auc_micro.append(test_ret['ave_auc_micro'])
    auc_macro.append(test_ret['ave_auc_macro'])
    auc_weighted.append(test_ret['ave_auc_weighted'])
    
    coverage_error.append(test_ret['coverage_error'])
    label_ranking_loss.append(test_ret['label_ranking_loss'])
#     pre1.append(test_ret['prec1'])
#     rec0.append(test_ret['rec0'])
#     rec1.append(test_ret['rec1'])
#     f1.append(test_ret['f1_score'])
    if (i+1)%100 == 0:
        print('%d/%d'%(i+1,K))

print('test_loss %.4f(%.4f)'%(np.mean(test_loss), np.std(test_loss)))
print('auc_micro %.4f(%.4f)'%(np.mean(auc_micro), np.std(auc_micro)))
print('auc_macro %.4f(%.4f)'%(np.mean(auc_macro), np.std(auc_macro)))
print('auc_weighted %.4f(%.4f)'%(np.mean(auc_weighted), np.std(auc_weighted)))

print('coverage_error %.4f(%.4f)'%(np.mean(coverage_error), np.std(coverage_error)))
print('label_ranking_loss %.4f(%.4f)'%(np.mean(label_ranking_loss), np.std(label_ranking_loss)))
# print('pre1 = PPV = precision %.4f(%.4f)'%(np.mean(pre1), np.std(pre1)))
# print('rec0 = TNR = specificity = selectivity %.4f(%.4f)'%(np.mean(rec0), np.std(rec0)))
# print('rec1 = recall = TPR = sensitivity %.4f(%.4f)'%(np.mean(rec1), np.std(rec1)))
# print('f1 %.4f(%.4f)'%(np.mean(f1), np.std(f1)))

100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
1000/1000
test_loss 0.1477(0.0162)
auc_micro 0.8503(0.0253)
auc_macro 0.8328(0.0279)
auc_weighted 0.8343(0.0296)
coverage_error 0.1959(0.0272)
label_ranking_loss 0.0226(0.0059)
